# Trénink jednoduchého modelu MLP

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# 1. Generování syntetického datasetu pro binární klasifikaci
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Definice jednoduché neuronové sítě
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Pro binární klasifikaci
])

# 3. Kompilace modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 4. Trénink modelu
model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32)

# 5. Vyhodnocení na testovací sadě
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# 6. Export do TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("mlp_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model byl úspěšně exportován do mlp_model.tflite")


Epoch 1/20
20/20 [==============================] - 0s 3ms/step - loss: 0.6341 - accuracy: 0.6016 - val_loss: 0.6188 - val_accuracy: 0.6375
Epoch 2/20
 1/20 [>.............................] - ETA: 0s - loss: 0.5761 - accuracy: 0.7500

2025-04-03 09:59:30.182875: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


20/20 [==============================] - 0s 828us/step - loss: 0.5502 - accuracy: 0.7453 - val_loss: 0.5507 - val_accuracy: 0.7375
Epoch 3/20
20/20 [==============================] - 0s 858us/step - loss: 0.4870 - accuracy: 0.8016 - val_loss: 0.4977 - val_accuracy: 0.8125
Epoch 4/20
20/20 [==============================] - 0s 845us/step - loss: 0.4351 - accuracy: 0.8391 - val_loss: 0.4577 - val_accuracy: 0.8438
Epoch 5/20
20/20 [==============================] - 0s 792us/step - loss: 0.3956 - accuracy: 0.8594 - val_loss: 0.4285 - val_accuracy: 0.8500
Epoch 6/20
20/20 [==============================] - 0s 759us/step - loss: 0.3661 - accuracy: 0.8687 - val_loss: 0.4109 - val_accuracy: 0.8562
Epoch 7/20
20/20 [==============================] - 0s 759us/step - loss: 0.3427 - accuracy: 0.8703 - val_loss: 0.3976 - val_accuracy: 0.8687
Epoch 8/20
20/20 [==============================] - 0s 705us/step - loss: 0.3288 - accuracy: 0.8734 - val_loss: 0.3888 - val_accuracy: 0.8687
Epoch 9/20
20/20 

2025-04-03 09:59:31.023486: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2025-04-03 09:59:31.023499: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2025-04-03 09:59:31.024007: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmpatsvx3nd
2025-04-03 09:59:31.024928: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-04-03 09:59:31.024932: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmpatsvx3nd
2025-04-03 09:59:31.026124: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2025-04-03 09:59:31.026406: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2025-04-03 09:59:31.041914: I tensorflow/cc/saved_model/loader.

# Rozpoznávání zvuku pomocí MFCC a MLP
Tento příklad načte zvukový soubor, extrahuje z něj MFCC příznaky a použije je pro klasifikaci (např. rozpoznání slova „yes“).

In [1]:
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 1. Načtení WAV souboru a výpočet MFCC
audio_path = 'audio_yes.wav'  # nahraď svou cestou
y, sr = librosa.load(audio_path, sr=16000)
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
mfcc_mean = np.mean(mfcc, axis=1)

# 2. Příprava dat (v tomto příkladu syntetická binární klasifikace yes/no)
X = np.array([mfcc_mean, mfcc_mean + np.random.normal(0, 0.1, 13)])
y = np.array([1, 0])  # 1 = yes, 0 = noise

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

# 3. Vytvoření MLP modelu
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(13,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15)

# 4. Vyhodnocení a export
loss, acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {acc:.2f}")

# 5. Export do .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("audio_mfcc_model.tflite", "wb") as f:
    f.write(tflite_model)

Epoch 1/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 2/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 3/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 4/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 5/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 6/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 7/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 8/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 9/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 10/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 11/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 1.0528e-27
Epoch 12/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step -

INFO:tensorflow:Assets written to: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmpwfvknglb/assets


Saved artifact at '/var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmpwfvknglb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 13), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  6247702288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6247702480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13386889744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13386888592: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743611205.781359 5586358 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743611205.781621 5586358 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-02 18:26:45.782275: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmpwfvknglb
2025-04-02 18:26:45.782590: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-02 18:26:45.782593: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmpwfvknglb
I0000 00:00:1743611205.783972 5586358 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-04-02 18:26:45.784136: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-02 18:26:45.796074: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folder

# Rozpoznávání obrazu pomocí MobileNetV2
Tento příklad používá MobileNetV2 jako základ a přetrénuje výstupní vrstvy pro vlastní třídy (např. klasifikace „cat“, „dog“). Funguje pro malé obrázky (např. z ESP32-CAM).

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

# 1. Načtení datasetu – použijeme ukázkový z TFDS
(train_ds, val_ds), ds_info = tfds.load('cats_vs_dogs', split=['train[:80%]', 'train[80%:]'],
                                        shuffle_files=True, as_supervised=True, with_info=True)

IMG_SIZE = 96

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0
    return image, label

train_ds = train_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

# 2. Model založený na MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=3)

# 3. Export do TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('mobilenet_image_model.tflite', 'wb') as f:
    f.write(tflite_model)


/opt/anaconda3/envs/CondaAiProjectTF210/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-02 11:40:59.232208: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Generating train examples...:  99%|█████████▊| 22957/23262 [00:02<00:00, 8607.68 examples/s]WARNING:absl:1738 images were corrupted and were skipped

                                                                                            
Shuffling /Users/milannovak/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteINK174/cats_vs_dogs-train.tfrecord*...:   0%|          | 0/23262 [00:00<?, ? examples/s]
Shuffling /Users/milannovak/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteINK174/cats_vs_dogs-train.tfrecord*...:  13%|█▎        | 2953/23262 [00:00<00:00, 29527.00 examples/s]
Shuffling /Users/milannovak/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteINK174/cats_vs_dogs-train.tfrecord*...:  25%|██▌       | 5906/23262 [00:00<00:00, 29368.81 examples/s]
Shuffling /Users/milannovak/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteINK174/cats_vs_dogs-train.tfrecord*...:  39%|███▊      | 8978/23262 [00:00<00:00, 29982.05 examples/s]
Shuffling /

Dataset cats_vs_dogs downloaded and prepared to /Users/milannovak/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Epoch 1/3


2025-04-02 11:41:13.870964: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


197/582 [=========>....................] - ETA: 14s - loss: 0.1952 - accuracy: 0.9204

...

582/582 [==============================] - ETA: 0s - loss: 0.1031 - accuracy: 0.9586

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


582/582 [==============================] - 27s 46ms/step - loss: 0.1031 - accuracy: 0.9586 - val_loss: 0.1238 - val_accuracy: 0.9514


INFO:tensorflow:Assets written to: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmptk52e41c/assets


INFO:tensorflow:Assets written to: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmptk52e41c/assets
2025-04-02 11:42:47.171429: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2025-04-02 11:42:47.171441: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2025-04-02 11:42:47.171733: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmptk52e41c
2025-04-02 11:42:47.183768: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-04-02 11:42:47.183779: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmptk52e41c
2025-04-02 11:42:47.222155: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2025-04-02 11:42:47.228994: I tensorflow/cc/saved_model/load